In [2]:
!pip install selenium

In [3]:
#자동 로그인
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
import random

# 세종대 에브리타임 자동 로그인

options = wd.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")

driver = wd.Chrome(executable_path='/usr/local/bin/chromedriver', options = options)
driver.maximize_window()


baseURL = 'https://sejong.everytime.kr'

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.get(baseURL)

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_css_selector('#account > a:nth-child(1)').click()

driver.find_element_by_name('userid').send_keys('gimsg12')
driver.find_element_by_name('password').send_keys('data')

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_xpath('//*[@class="submit"]/input').click()

In [15]:
lecture = pd.read_csv("data/lecture_data.csv")
lecture.head()

Unnamed: 0   url_id  lec_num pro_name   star
0           0   535020     8802      이재교   86.6
1           1  1950200     9600      김희정   96.4
2           2  2099759     9608      한명관  100.0
3           3  1788599     3706       이원   95.0
4           4  1788598     9607      한명관   75.0

In [5]:
from bs4 import BeautifulSoup

In [48]:
import re
#강의 -> 강의평가
lecture_review_list=[]
url_id_list = lecture.url_id.astype(str) 

for url_id in url_id_list[2:15]:
    #강의 페이지로 넘어가기
    url = "https://everytime.kr/lecture/view/"+url_id
    rand_value = random.uniform(2,5)
    time.sleep(rand_value)
    driver.get(url)
    
    #하나의 강의 평가 페이지 bs로 데이터 수집
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser') #정적텍스트 분석을 사용하는 BS이 selenium보다 빠름 -> 페이지 접근에는 동적 수집, 데이터 수집은 정적수집적
    try:
        lecture_name = soup.select_one("#container > div.side.head > h2").get_text()
        professor_name = soup.select_one("#container > div.side.head > p:nth-child(2) > span").get_text()
        review_list = soup.select("article")
        print(lecture_name, professor_name)
    except: #왜 안되는지 모르게씀...;;;
        pass
    #강의리뷰 
    key = int(url_id)*10
    for review in review_list:
        rate_html = str(review.select_one('article > p.rate > span > span'))
        rate = re.findall('\d+', rate_html) #숫자만 (리스트형태로 return되는데 [0]하면 오류뜸...)
        
        semester = review.select_one('article > p.info > span').get_text()
        semester = int(str(semester)[0:2] + str(semester)[4])
    
        text = review.select_one('article > p.text') 
        text = str(text)[16:-4]
        
        key += 1
        lecture_review_list.append({
            "key"            :key,
            "lecture_name"   : lecture_name,
            "professor_name" : professor_name,
            "rate"           : rate,
            "semester"       : semester,
             "text"          : text
           }
        )

형법총론 한명관
행정법1 이원
형사소송법 한명관
노동법 조남택
민사소송법2 이재교
졸업연구및진로1 이재교
교육학의탐구 이수정
교육철학의이해 김영래
교수학습이론 이인숙
신입생세미나 주경희
한국한문학의이해 이현주
한국현대소설론 최진옥
국어교육론 정영교


In [49]:
for review in lecture_review_list:
    print(review)
    print()

{'key': 20997591, 'lecture_name': '형법총론', 'professor_name': '한명관', 'rate': ['100'], 'semester': 201, 'text': '조문 위주로 설명해주시며 특히 학생들이 어려워하는<br/>포인트를 정확하게 집어서 상세하게 알려주심<br/>과제 하나 빼먹었는데 a 받아서 그저 감사할 따름입니다<br/>시험도 크게 어렵지 않아서 법학부 수업 중에는 학점<br/>잘 따기 좋은 강좌 상위권에 들 것 같습니다'}

{'key': 20997592, 'lecture_name': '형법총론', 'professor_name': '한명관', 'rate': ['100'], 'semester': 201, 'text': '학생들이 어떻게 하면 이해하기 쉬울까 고민하고 강의를 준비하시는 느낌입니다.<br/>교재도 안썼고, 교수님이 직접 준비하신 교안으로  수업을 진행했는데,<br/>교재보고 공부하기보단 조문을 중심으로 중요한 내용들을 이해하는게 먼저라고 하시던게 기억에 남네요.<br/>과제도 전혀 어렵지 않은 수준이었습니다. 오픈북 시험이라 시험도 교안 중심으로 옮겨쓰거나 그런 것들 중심으로 나왔구요. 학점을 잘 주시는지는 잘 모르겠네요... 전 나름 다 쓴 것 같은데 에이가 나와부려서 흑'}

{'key': 20997593, 'lecture_name': '형법총론', 'professor_name': '한명관', 'rate': [], 'semester': 201, 'text': '이 정보를 확인하기 위해 포인트를 사용하시겠습니까. 포인트를 사용하시겠습니까. 이 시험 정보를 확인하기 '}

{'key': 17885991, 'lecture_name': '행정법1', 'professor_name': '이원', 'rate': ['100'], 'semester': 201, 'text': '교수님이 엄청 친절하고 좋으십니다. 공부해야할 양이 많긴 했는데 과제도 간단한 레포트 정도였고 크게 힘들지 않았던 수업입니다.'}

{'key

In [50]:
review_data = pd.DataFrame(lecture_review_list)
review_data.to_csv('data/review_data.csv',index=False,encoding='utf-8-sig') #한글 인코딩 깨질때
len(review_data)

62

In [51]:
review_data.head()

key lecture_name professor_name   rate  semester  \
0  20997591         형법총론            한명관  [100]       201   
1  20997592         형법총론            한명관  [100]       201   
2  20997593         형법총론            한명관     []       201   
3  17885991         행정법1             이원  [100]       201   
4  17885992         행정법1             이원   [80]       201   

                                                text  
0  조문 위주로 설명해주시며 특히 학생들이 어려워하는<br/>포인트를 정확하게 집어서 ...  
1  학생들이 어떻게 하면 이해하기 쉬울까 고민하고 강의를 준비하시는 느낌입니다.<br/...  
2  이 정보를 확인하기 위해 포인트를 사용하시겠습니까. 포인트를 사용하시겠습니까. 이 ...  
3  교수님이 엄청 친절하고 좋으십니다. 공부해야할 양이 많긴 했는데 과제도 간단한 레포...  
4  (온라인 기준)<br/>교수님 정말 열심히 하심<br/>과제도 엄청 쉽고 <br/>...

In [ ]:
baseURL = 'https://everytime.kr/377390'

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.get(baseURL)

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_css_selector('#account > a:nth-child(1)').click()

driver.find_element_by_name('userid').send_keys('gimsg12')
driver.find_element_by_name('password').send_keys('data')

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_xpath('//*[@class="submit"]/input').click()